In [ ]:
import subprocess
import threading
import ipywidgets as widgets
from IPython.display import display
import os
# .exe 파일이 있는 폴더 경로와 파일 이름
exe_folder_path = "C:\\Users\\Seungmi\\Desktop\\May\\S-SDKZ50-008BF-ALLIN\Module\\Win\\Sample Program\\Type0025_CtrlSample_Win\\win\\build\\x64\\Release"
exe_file_name = "Type0025Ctrl.exe"

# .exe 파일이 있는 폴더로 작업 디렉토리를 변경
os.chdir(exe_folder_path)
exe_path = os.path.join(exe_folder_path, exe_file_name)


output_area = widgets.Output()
input_box = widgets.Text(placeholder="Enter input for the program")
send_button = widgets.Button(description="Send")
stop_button = widgets.Button(description="Stop Program", button_style="danger")

# 위젯 UI 표시
display(input_box, send_button, stop_button, output_area)

# subprocess를 통해 .exe 파일 실행 및 초기 입력 전달
process = subprocess.Popen([exe_file_name],
                           stdin=subprocess.PIPE,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           text=True)

# 출력을 실시간으로 읽어오는 스레드 종료 플래그
stop_thread = False

# 출력을 읽어와 출력 영역에 표시하는 함수
def read_output():
    while not stop_thread:
        output = process.stdout.readline()
        if output:
            with output_area:
                print(output.strip())  # 프로그램의 출력을 실시간으로 표시
        elif process.poll() is not None:  # 프로세스가 종료된 경우
            break

# 출력을 읽는 스레드를 생성하여 실행
output_thread = threading.Thread(target=read_output)
output_thread.start()

# Send 버튼 클릭 시 실행할 함수 정의
def on_send_button_clicked(b):
    user_input = input_box.value
    input_box.value = ""  # 입력 필드를 비웁니다.
    if process.poll() is None:  # 프로그램이 실행 중인지 확인
        process.stdin.write(user_input + "\n")
        process.stdin.flush()  # 입력을 강제로 전달
        with output_area:
            print("Sent input:", user_input)

# Stop 버튼 클릭 시 실행할 함수 정의
def on_stop_button_clicked(b):
    global stop_thread
    if process.poll() is None:  # 프로그램이 실행 중이면 종료
        process.terminate()
        stop_thread = True  # 스레드를 종료하도록 플래그 설정
        output_thread.join()  # 스레드가 종료될 때까지 기다림
        with output_area:
            print("Program terminated.")

# 버튼 클릭 이벤트 연결
send_button.on_click(on_send_button_clicked)
stop_button.on_click(on_stop_button_clicked)